# LAB 1
In this notebook, we use the following modules `numpy` and `minotaur_maze`. The latter is a home made module, where all the solutions to the questions are implemented. 

In [ ]:
import numpy as np
import minoutaur_maze as mz
import matplotlib.pyplot as plt

# Problem 1: The Maze and the Random Minotaur

The objective of problem 1 is to solve the shortest path problem in a maze. We start first by describing the maze as a numpy array. 

In [ ]:
# starting positions
thomas_st = (0,0)
minotaur_st = (6,5)

# no need to pick key (we have it from the beginning)
key_st = (1,)

# starting state
start = (thomas_st + minotaur_st + key_st)

# Description of the maze as a numpy array
maze = np.array([
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 1, 1],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 1, 2, 0, 0]
])
# with the convention 
# 0 = empty cell
# 1 = obstacle
# 2 = exit of the Maze

The method `maze.draw_maze()` helps us draw the maze given its numpy array discription.  

In [ ]:
mz.draw_maze(maze)

## THE MINOTAUR CANNOT WAIT   

In [ ]:
# Create an environment maze
minotaur_can_wait = False
key_needed = False
env_1 = mz.MinotaurMaze(maze,minotaur_can_wait,key_needed)
#env_1.show()
print('# states = ', env_1.n_states)
print('# actions = ', env_1.n_actions)

###  Dynamic Programming 

Before solving the MDP problem, recall that the finite horizon objective function is 
$$
    \mathbb{E} \Big[ \sum_{t=0}^T r(s_t, a_t) \Big],
$$
where $T$ is the horizon.
Recall the Bellman equation 
\begin{equation}
\forall s \in \mathcal{S} \qquad  V(s) = \max_{a \in \mathcal{A}} \Big\lbrace r(s,a) + \sum_{s' \in \mathcal{S}} \mathbb{P}(s'\vert s,a) V(s') \Big\rbrace
\end{equation}
The dynamic programming solution for the finite horizon MDP problem consists of solving the above backward recursion. The method `maze.dynamic_programming` achieves this. 
> **Note:** To find the optimal path, it is enough to set the time horizon $T = 10$. Indeed, looking at the maze one can see that the player needs at least 10 steps to attain the exit $B$, if her starting position is at $A$. In fact if you set the time horizon less than 10, you will see that you do not find the optimal path.



In [ ]:
# Finite horizon
horizon = 17
# Solve the MDP problem with dynamic programming 
V, policy = mz.dynamic_programming(env_1,horizon)

In [ ]:
# Simulate the best path (reacting to random minotaur actions)
method = 'DynProg'
path, victory_prob = env_1.simulate(start, policy, method)

In [ ]:
# Animate path 
show_arrows = True
fps = 1
mz.animate_solution(maze, path, show_arrows, fps)

In [ ]:
cell_probs = env_1.minotaur_cell_probs(maze,minotaur_st,horizon)

In [ ]:
t = horizon
plt.imshow(cell_probs[t,:,:])

In [ ]:
# Victory prob if the path is fixed
fixed_path_vict_prob = env_1.fixed_path_vict_prob(maze,path,cell_probs)
fixed_path_vict_prob

### POLICY THAT MAXIMIZES PROB. OF LEAVING FOR T = 20

In [ ]:
# Finite horizon
horizon = 20
# Solve the MDP problem with dynamic programming 
V, policy = mz.dynamic_programming(env_1,horizon)

In [ ]:
# Simulate several paths and compute victory prob
method = 'DynProg'
N = 5000
paths_dict = {}
probs_dict = {}
probs = []
for n in range(N) :
    path, victory_prob = env_1.simulate(start, policy, method)
    paths_dict[n] = path
    probs_dict[n] = victory_prob
    probs.append(victory_prob)

probs.sort()
print('# SIMULATIONS = ', N)
print('Mean Victory Prob = ', sum(probs)/len(probs))

### POLICY THAT MAXIMIZES PROB FOR T = {1, ..., 30} AND PROBABILITY OF LEAVING THE MAZE FOR EACH T


In [ ]:
T_min = 1
T_max = 30
horizons = range(T_min,T_max+1)
Vs, policys = [], []
for horizon in horizons :
    V, policy = mz.dynamic_programming(env_1,horizon)
    Vs.append(V)
    policys.append(policy)

In [ ]:
method = 'DynProg'
N = 5000
# Simulate N paths for each time horizon
mean_victory_probs = []
for i in range(T_max) :
    probs = []
    for n in range(N) :
        path, victory_prob = env_1.simulate(start, policys[i], method)
        probs.append(victory_prob)
    # Compute mean victory prob
    mean_victory_probs.append(sum(probs)/len(probs))

In [ ]:
# Plot mean victory prob for each horizon
plt.plot(horizons, mean_victory_probs, 'ro')

## THE MINOTAUR CAN WAIT

In [ ]:
# Create an environment maze
minotaur_can_wait = True
key_needed = False
env_2 = mz.MinotaurMaze(maze,minotaur_can_wait,key_needed)
#env_2.show()
print('# states = ', env_2.n_states)
print('# actions = ', env_2.n_actions)

###  Dynamic Programming 

Before solving the MDP problem, recall that the finite horizon objective function is 
$$
    \mathbb{E} \Big[ \sum_{t=0}^T r(s_t, a_t) \Big],
$$
where $T$ is the horizon.
Recall the Bellman equation 
\begin{equation}
\forall s \in \mathcal{S} \qquad  V(s) = \max_{a \in \mathcal{A}} \Big\lbrace r(s,a) + \sum_{s' \in \mathcal{S}} \mathbb{P}(s'\vert s,a) V(s') \Big\rbrace
\end{equation}
The dynamic programming solution for the finite horizon MDP problem consists of solving the above backward recursion. The method `maze.dynamic_programming` achieves this. 
> **Note:** To find the optimal path, it is enough to set the time horizon $T = 10$. Indeed, looking at the maze one can see that the player needs at least 10 steps to attain the exit $B$, if her starting position is at $A$. In fact if you set the time horizon less than 10, you will see that you do not find the optimal path.


In [ ]:
# Finite horizon
horizon = 20
# Solve the MDP problem with dynamic programming 
V, policy = mz.dynamic_programming(env_2,horizon)

In [ ]:
# Simulate the best path (reacting to random minotaur actions)
method = 'DynProg'
path, victory_prob = env_2.simulate(start, policy, method)

In [ ]:
# Animate path 
show_arrows = True
fps = 1
mz.animate_solution(maze, path, show_arrows, fps)

In [ ]:
# Cell probabilities
cell_probs = env_2.minotaur_cell_probs(maze,minotaur_st,horizon)

In [ ]:
t = horizon
plt.imshow(cell_probs[t,:,:])

In [ ]:
# Victory prob if the path is fixed
fixed_path_vict_prob = env_2.fixed_path_vict_prob(maze,path,cell_probs)
fixed_path_vict_prob

### POLICY THAT MAXIMIZES PROB FOR T = {1, ..., 30} AND PROBABILITY OF LEAVING THE MAZE FOR EACH T

In [ ]:
T_min = 1
T_max = 30
horizons = range(T_min,T_max+1)
Vs, policys = [], []
for horizon in horizons :
    V, policy = mz.dynamic_programming(env_2,horizon)
    Vs.append(V)
    policys.append(policy)

In [ ]:
method = 'DynProg'
N = 5000
# Simulate N paths for each time horizon
mean_victory_probs = []
for i in range(T_max) :
    probs = []
    for n in range(N) :
        path, victory_prob = env_2.simulate(start, policys[i], method)
        probs.append(victory_prob)
    # Compute mean victory prob
    mean_victory_probs.append(sum(probs)/len(probs))

In [ ]:
# Plot mean victory prob for each horizon
plt.plot(horizons, mean_victory_probs, 'ro')

### GEOMETRICALLY DISTRIBUTED LIFE (TIME HORIZON) WITH MEAN 30

In [ ]:
# Finite horizon
gamma = 29/30
epsilon = 0.0001
# Solve the MDP problem with value iteration
V, policy = mz.value_iteration(env_1,gamma,epsilon)

In [ ]:
# Simulate the best path (reacting to random minotaur actions)
method = 'ValIter'
path, victory_prob = env_1.simulate(start, policy, method)

In [ ]:
# Animate path 
show_arrows = True
fps = 1
mz.animate_solution(maze, path, show_arrows, fps)

### PROBABILITY OF GETTING OUT ALIVE USING THE VALUE ITERATION POLICY

### If the minotaur cannot wait

In [ ]:
E_T_max = 62
E_T_min = 2
E_Ts = range(E_T_min,E_T_max,2)
Vs, policys = [], []
gammas = []
epsilon = 0.0001
for E_T in E_Ts :
    gamma = (E_T-1)/E_T
    gammas.append(gamma)
    V, policy = mz.value_iteration(env_1,gamma,epsilon)
    Vs.append(V)
    policys.append(policy)

In [ ]:
method = 'ValIter'
N = 5000
# Simulate N paths for each gamma
mean_victory_probs = []
for i in range(len(gammas)) :
    probs = []
    for n in range(N) :
        path, victory_prob = env_1.simulate(start, policys[i], method)
        probs.append(victory_prob)
    # Compute mean victory prob
    mean_victory_probs.append(sum(probs)/len(probs))

In [ ]:
# Plot mean victory prob for each gamma
plt.plot(gammas, mean_victory_probs, 'ro')

### If the minotaur can wait

In [ ]:
E_T_max = 62
E_T_min = 2
E_Ts = range(E_T_min,E_T_max,2)
Vs, policys = [], []
gammas = []
epsilon = 0.0001
for E_T in E_Ts :
    gamma = (E_T-1)/E_T
    gammas.append(gamma)
    V, policy = mz.value_iteration(env_2,gamma,epsilon)
    Vs.append(V)
    policys.append(policy)

In [ ]:
method = 'ValIter'
N = 5000
# Simulate N paths for each gamma
mean_victory_probs = []
for i in range(len(gammas)) :
    probs = []
    for n in range(N) :
        path, victory_prob = env_2.simulate(start, policys[i], method)
        probs.append(victory_prob)
    # Compute mean victory prob
    mean_victory_probs.append(sum(probs)/len(probs))

In [ ]:
# Plot mean victory prob for each gamma
plt.plot(gammas, mean_victory_probs, 'ro')

## IF WE NEED THE KEY TO SCAPE

In [ ]:
# starting positions
thomas_st = (0,0)
minotaur_st = (6,5)

# we need to pick the key (we dont have it from the beginning)
key_st = (0,)

# starting state
start = (thomas_st + minotaur_st + key_st)

# Description of the maze as a numpy array
maze = np.array([
    [0, 0, 1, 0, 0, 0, 0, 3],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 1, 1],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 1, 2, 0, 0]
])
# with the convention 
# 0 = empty cell
# 1 = obstacle
# 2 = exit of the Maze
# 3 = key

In [ ]:
# Create an environment maze
minotaur_can_wait = False
key_needed = True
env_3 = mz.MinotaurMaze(maze,minotaur_can_wait,key_needed)
#env_1.show()
print('# states = ', env_3.n_states)
print('# actions = ', env_3.n_actions)

## Solving with Value Iteration

In [ ]:
# Finite horizon
gamma = 59/60
epsilon = 0.0001
# Solve the MDP problem with value iteration
V, policy = mz.value_iteration(env_3,gamma,epsilon)

In [ ]:
# Simulate the best path (reacting to random minotaur actions)
method = 'ValIter'
path, victory_prob = env_3.simulate(start, policy, method)

In [ ]:
# Animate path 
show_arrows = True
fps = 10
mz.animate_solution(maze, path, show_arrows, fps)

## Solving with Q-Learning

### Fixing learning rate and modifying epsilon

### $\epsilon = 0.5$

In [ ]:
# T is Geo(1-gamma)
gamma = 59/60
# Initial exploration prob.
epsilon = 0.5
Q, policy, init_Vs = mz.qLearning(env_3, start, gamma, epsilon, 
    n_episodes=50000,
    max_iters=200,
    decay_delta=0.
)

In [ ]:
# Plot tot reward for each episode
plt.plot(range(len(init_Vs)), init_Vs)

In [ ]:
# Simulate the best path (reacting to random minotaur actions)
method = 'Q-Learning'
path, victory_prob = env_3.simulate(start, policy, method)
# Animate path 
show_arrows = True
fps = 10
mz.animate_solution(maze, path, show_arrows, fps)

### $\epsilon = 0.2$

In [ ]:
# T is Geo(1-gamma)
gamma = 59/60
# Initial exploration prob.
epsilon = 0.2
Q, policy, init_Vs = mz.qLearning(env_3, start, gamma, epsilon, 
    n_episodes=50000,
    max_iters=200,
    decay_delta=0.
)

In [ ]:
# Plot tot reward for each episode
plt.plot(range(len(init_Vs)), init_Vs)

In [ ]:
# Simulate the best path (reacting to random minotaur actions)
method = 'Q-Learning'
path, victory_prob = env_3.simulate(start, policy, method)
# Animate path 
show_arrows = True
fps = 10
mz.animate_solution(maze, path, show_arrows, fps)

### Fixing $\epsilon$ and modifying learning rate

### lr_alpha = 0.55

In [ ]:
# T is Geo(1-gamma)
gamma = 59/60
# Initial exploration prob.
epsilon = 0.35
Q, policy, init_Vs = mz.qLearning(env_3, start, gamma, epsilon, 
    n_episodes=50000, 
    max_iters=200, 
    decay_delta=0.,
    lr_alpha = 0.55
)

In [ ]:
# Plot tot reward for each episode
plt.plot(range(len(init_Vs)), init_Vs)

In [ ]:
# Simulate the best path (reacting to random minotaur actions)
method = 'Q-Learning'
path, victory_prob = env_3.simulate(start, policy, method)
# Animate path 
show_arrows = True
fps = 10
mz.animate_solution(maze, path, show_arrows, fps)

### lr_alpha = 0.8

In [ ]:
# T is Geo(1-gamma)
gamma = 59/60
# Initial exploration prob.
epsilon = 0.35
Q, policy, init_Vs = mz.qLearning(env_3, start, gamma, epsilon, 
    n_episodes=50000, 
    max_iters=200, 
    decay_delta=0.,
    lr_alpha = 0.8
)

In [ ]:
# Plot tot reward for each episode
plt.plot(range(len(init_Vs)), init_Vs)

In [ ]:
# Simulate the best path (reacting to random minotaur actions)
method = 'Q-Learning'
path, victory_prob = env_3.simulate(start, policy, method)
# Animate path 
show_arrows = True
fps = 10
mz.animate_solution(maze, path, show_arrows, fps)

## Solving with SARSA

### Fixing learning rate and modifying epsilon

### $\epsilon = 0.2$

In [ ]:
# T is Geo(1-gamma)
gamma = 59/60
# Initial exploration prob.
epsilon = 0.2
Q, policy, init_Vs = mz.sarsa(env_3, start, gamma, epsilon, 
    n_episodes=50000,
    max_iters=200,
    decay_delta=0.
)

In [ ]:
# Plot tot reward for each episode
plt.plot(range(len(init_Vs)), init_Vs)

In [ ]:
# Simulate the best path (reacting to random minotaur actions)
method = 'SARSA'
path, victory_prob = env_3.simulate(start, policy, method)
# Animate path 
show_arrows = True
fps = 10
#mz.animate_solution(maze, path, show_arrows, fps)

### $\epsilon = 0.1$

In [ ]:
# T is Geo(1-gamma)
gamma = 59/60
# Initial exploration prob.
epsilon = 0.1
Q, policy, init_Vs = mz.sarsa(env_3, start, gamma, epsilon, 
    n_episodes=50000,
    max_iters=200,
    decay_delta=0.
)

In [ ]:
# Plot tot reward for each episode
plt.plot(range(len(init_Vs)), init_Vs)

In [ ]:
# Simulate the best path (reacting to random minotaur actions)
method = 'SARSA'
path, victory_prob = env_3.simulate(start, policy, method)
# Animate path 
show_arrows = True
fps = 10
#mz.animate_solution(maze, path, show_arrows, fps)

### Decreasing $\epsilon$ by $\frac{1}{e^\delta}$

In [ ]:
# T is Geo(1-gamma)
gamma = 59/60
# Initial exploration prob.
epsilon = 1
Q, policy, init_Vs = mz.sarsa(env_3, start, gamma, epsilon, 
    n_episodes=50000,
    max_iters=200,
    decay_delta=0.75 # alpha lower than delta
)

In [ ]:
# Plot tot reward for each episode
plt.plot(range(len(init_Vs)), init_Vs)

In [ ]:
# T is Geo(1-gamma)
gamma = 59/60
# Initial exploration prob.
epsilon = 1
Q, policy, init_Vs = mz.sarsa(env_3, start, gamma, epsilon, 
    n_episodes=50000,
    max_iters=200,
    decay_delta=0.55 # alpha greater than delta
)

In [ ]:
# Plot tot reward for each episode
plt.plot(range(len(init_Vs)), init_Vs)

## Prob. of winning with Q-LEARNING and SARSA policies

### With Q-Learning

In [ ]:
# T is Geo(1-gamma)
gamma = 59/60
# Initial exploration prob.
epsilon = 0.2
Q, q_policy, q_init_Vs = mz.qLearning(env_3, start, gamma, epsilon, 
    n_episodes=50000,
    max_iters=200,
    decay_delta=0.
)

In [ ]:
# Plot tot reward for each episode
plt.plot(range(len(q_init_Vs)), q_init_Vs)

In [ ]:
# Simulate several paths and compute victory prob
method = 'Q-Learning'
N = 5000
paths_dict = {}
probs_dict = {}
probs = []
for n in range(N) :
    path, victory_prob = env_3.simulate(start, q_policy, method)
    paths_dict[n] = path
    probs_dict[n] = victory_prob
    probs.append(victory_prob)

probs.sort()
print('# SIMULATIONS = ', N)
print('Mean Victory Prob = ', sum(probs)/len(probs))

### With SARSA

In [ ]:
# T is Geo(1-gamma)
gamma = 59/60
# Initial exploration prob.
epsilon = 0.2
Q, s_policy, s_init_Vs = mz.sarsa(env_3, start, gamma, epsilon, 
    n_episodes=50000,
    max_iters=200,
    decay_delta=0.
)

In [ ]:
# Plot tot reward for each episode
plt.plot(range(len(s_init_Vs)), s_init_Vs)

In [ ]:
# Simulate several paths and compute victory prob
method = 'SARSA'
N = 5000
paths_dict = {}
probs_dict = {}
probs = []
for n in range(N) :
    path, victory_prob = env_3.simulate(start, s_policy, method)
    paths_dict[n] = path
    probs_dict[n] = victory_prob
    probs.append(victory_prob)

probs.sort()
print('# SIMULATIONS = ', N)
print('Mean Victory Prob = ', sum(probs)/len(probs))